# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
import pandas as pd

In [2]:
client = MongoClient("localhost:27017")

In [3]:
client.list_database_names()

['admin', 'config', 'ironhack', 'local']

In [4]:
db = client.get_database("ironhack")

In [5]:
db.list_collection_names()

['restaurants', 'companies']

In [6]:
c = db.get_collection("companies")

In [7]:
pdc = list(c.find_one())

In [8]:
df = pd.DataFrame(pdc)
df

,0
0,_id
1,name
2,permalink
3,crunchbase_url
4,homepage_url
5,blog_url
6,blog_feed_url
7,twitter_username
8,category_code
9,number_of_employees


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [9]:
filtro = {"name": "Babelgum"}
proj = {"_id": 0, "name": 1}

In [10]:
list(c.find(filtro, proj))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [11]:
filtro = {"number_of_employees": {"$gt": 5000}}
proj = {"_id": 0, "name": 1, "number_of_employees": 1}

In [12]:
list(c.find(filtro,proj).limit(5).sort("number_of_employees",-1))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [13]:
#filtro = {"number_of_employees": {"$gt": 5000}}
proj = {"_id": 0, "name": 1, "founded_year": 1}
query = {"$and": [{"founded_year": {"$gte":2000}},{"founded_year": {"$lte":2005}}]}

In [14]:
list(c.find(query,proj).limit(5))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [15]:
proj = {"_id": 0, "name": 1, "ipo": 1}
query = {"$and": [{"founded_year": {"$lt":2010}},{"ipo.valuation_amount": {"$lt":100000000}}]}

In [16]:
list(c.find(query,proj).limit(2))

[{'name': 'Vringo',
  'ipo': {'valuation_amount': 11003200,
   'valuation_currency_code': 'USD',
   'pub_year': 2010,
   'pub_month': 6,
   'pub_day': 22,
   'stock_symbol': 'VRNG'}},
 {'name': 'Pironet',
  'ipo': {'valuation_amount': 44000000,
   'valuation_currency_code': 'EUR',
   'pub_year': 2000,
   'pub_month': 2,
   'pub_day': 22,
   'stock_symbol': 'PNG'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [17]:
# Your Code
proj = {"_id": 0, "name": 1, "number_of_employees": 1}
query = {"$and": [{"founded_year": {"$lt":2005}},{"number_of_employees": {"$lt":1000}}]}

In [18]:
list(c.find(query,proj).limit(10).sort("number_of_employees", -1))

[{'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'Webmetrics', 'number_of_employees': 900},
 {'name': 'SonicWALL', 'number_of_employees': 900},
 {'name': 'Cornerstone OnDemand', 'number_of_employees': 881},
 {'name': 'Buongiorno', 'number_of_employees': 800},
 {'name': 'Cvent', 'number_of_employees': 800},
 {'name': 'ZoomInfo', 'number_of_employees': 800}]

### 6. All the companies that don't include the `partners` field.

In [55]:
proj = {"_id": 0, "name": 1, "partners": 1}
query = {"partners": {"$exists": False}}

In [56]:
# Your Code
list(c.find(query,proj).limit(5))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [57]:
proj = {"_id": 0, "name": 1, "category_code": 1}
query = {"category_code" : {'$type': 10}}

In [58]:
# Your Code
list(c.find(query,proj).limit(5))

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [65]:
filtro = {"$and": [{"number_of_employees": {"$gte":100}},{"number_of_employees": {"$lte":1000}}]}
proj = {"_id": 0, "name": 1, "number_of_employees": 1}

In [67]:
# Your Code
list(c.find(filtro, proj).limit(5))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [95]:
filtro = {"ipo.valuation_amount": {"$ne": None}}
proj = {"_id": 0, "name": 1, "ipo.valuation_amount": 1}

In [99]:
# Your Code
list(c.find(filtro, proj).sort("ipo.valuation_amount", -1).limit(5))

[{'name': 'GREE', 'number_of_employees': 700},
 {'name': 'Facebook', 'number_of_employees': 5299},
 {'name': 'Twitter', 'number_of_employees': 1300},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'LinkedIn', 'number_of_employees': 4800}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [97]:
proj = {"_id": 0, "name": 1, "number_of_employees": 1}
filtro = {"number_of_employees": {"$gt": 1}}

In [100]:
# Your Code
list(c.find(filtro, proj).sort("number_of_employees", -1).limit(5))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [104]:
# Your Code
proj = {"_id": 0, "name": 1}
filtro = {"founded_month": {"$gte" : 7}}

In [105]:
list(c.find(filtro, proj).limit(1000))

[{'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Omnidrive'},
 {'name': 'eBay'},
 {'name': 'Joost'},
 {'name': 'Plaxo'},
 {'name': 'Powerset'},
 {'name': 'Kyte'},
 {'name': 'Thoof'},
 {'name': 'Jingle Networks'},
 {'name': 'LifeLock'},
 {'name': 'Wesabe'},
 {'name': 'SmugMug'},
 {'name': 'Google'},
 {'name': 'Skype'},
 {'name': 'Pando Networks'},
 {'name': 'Ikan'},
 {'name': 'delicious'},
 {'name': 'AllPeers'},
 {'name': 'Wize'},
 {'name': 'SellABand'},
 {'name': 'iContact'},
 {'name': 'MeeVee'},
 {'name': 'blinkx'},
 {'name': 'Zlio'},
 {'name': 'Yelp'},
 {'name': 'Jaiku'},
 {'name': 'Yapta'},
 {'name': 'Fleck'},
 {'name': 'SideStep'},
 {'name': 'RockYou'},
 {'name': 'Instructables'},
 {'name': 'Netvibes'},
 {'name': 'Swivel'},
 {'name': 'Slide'},
 {'name': 'CrowdVine'},
 {'name': 'TheFind'},
 {'name': 'Sway'},
 {'name': 'CornerWorld'},
 {'name': 'FlickIM'},
 {'name': 'Terabitz'},
 {'name': 'Kaboodle'},
 {'name': 'GigaOM'},
 {'name': 'Redfin'},
 {'name': 'Wink'},

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [106]:
# Your Code
proj = {"_id": 0, "name": 1}
filtro = {"$and": [{"founded_year": {"$lt": 2000}},{"acquisition.price_amount": {"$gt":10000000}}]}

In [108]:
list(c.find(filtro, proj).limit(5))

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [110]:
# Your Code
proj = {"_id": 0, "name": 1, "acquisition": 1}
filtro = {"acquisition.acquired_year": {"$gt":2010}}

In [112]:
list(c.find(filtro, proj).sort("acquisition.price_amount", -1).limit(5))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name': 'LS

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [118]:
# Your Code
proj = {"_id": 0, "name": 1, "founded_year": 1}
filtro = {"founded_year": {"$ne": None}}

In [119]:
list(c.find(filtro, proj).sort("founded_year", 1).limit(5))

[{'name': 'Alstrasoft', 'founded_year': 1800},
 {'name': 'SmallWorlds', 'founded_year': 1800},
 {'name': 'US Army', 'founded_year': 1800},
 {'name': 'DuPont', 'founded_year': 1802},
 {'name': 'Bachmann Industries', 'founded_year': 1833}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [123]:
# Your Code
proj = {"_id": 0, "name": 1, "acquisition.price_amount": 1}
filtro = {"founded_day": {"$lte": 7}}

In [124]:
list(c.find(filtro, proj).sort("acquisition.price_amount", -1).limit(10))

[{'name': 'Netscape', 'acquisition': {'price_amount': 4200000000}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos', 'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba', 'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger', 'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems', 'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense', 'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee', 'acquisition': {'price_amount': 321000000}},
 {'name': 'Vitrue', 'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [132]:
# Your Code
proj = {"_id": 0, "name": 1}
filtro = {"$and": [{"category_code": "web"},{"number_of_employees": {"$gt":4000}}]}

In [133]:
list(c.find(filtro, proj).sort("number_of_employees", -1).limit(5))

[{'name': 'Experian'},
 {'name': 'eBay'},
 {'name': 'Yahoo!'},
 {'name': 'Los Angeles Times Media Group'},
 {'name': 'Rakuten'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [137]:
# Your Code
proj = {"_id": 0, "name":1}
filtro = {"$and" : [{"acquisition.price_amount":{"$gt":10000000}}, {"acquisition.price_currency_code": "EUR"}]}

In [138]:
list(c.find(filtro, proj).limit(5))

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [145]:
# Your Code
proj = {"_id": 0, "name": 1, "acquisition": 1}
filtro = {"acquisition.acquired_month": {"$lte" : 3}}

In [146]:
list(c.find(filtro, proj).limit(10))

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [148]:
# Your Code
proj = {"_id": 0, "name":1}
filtro = {"$and" : [{"founded_year": {"$lte": 2010}}, {"founded_year": {"$gte": 2000}}, {"acquisition.acquired_year": {"$gt" : 2011}}]}

In [149]:
list(c.find(filtro, proj).limit(5))

[{'name': 'Wetpaint'},
 {'name': 'Digg'},
 {'name': 'Geni'},
 {'name': 'blogTV'},
 {'name': 'Revision3'}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [195]:
# Intentado pero no me sale...
proj = {"_id": 0, "name":1, "deadpooled_year":1}
filtro = {"deadpooled_year": {"$gt": 3}}
filtro = {"$and" : [{"deadpooled_year": {"$gt": 3}}, {"deadpooled_year": {"$regex": "\d{1}"}}]}

In [196]:
list(c.find(filtro, proj).limit(5))

[]